In [116]:
import pandas as pd 
import numpy as np
import re
from matplotlib import pyplot as plt
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation

In [2]:
train = pd.read_csv('train_E6oV3lV.csv')
test = pd.read_csv('test_tweets_anuFYb8.csv')
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 3 columns):
id       31962 non-null int64
label    31962 non-null int64
tweet    31962 non-null object
dtypes: int64(2), object(1)
memory usage: 749.2+ KB


In [4]:
y_train = train.label.values

In [5]:
from keras.preprocessing import text, sequence
from keras.layers import Bidirectional, LSTM, GRU, Dense, Flatten, Input, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.layers import Embedding, SpatialDropout1D, Dropout, BatchNormalization, Conv1D, concatenate
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from keras.backend import backend as K

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [6]:
X_train = train.tweet.values
X_test = test.tweet.values

In [7]:
max_feature = 10000
maxlen = 150
embedding_size = 50

In [8]:
tokenize = text.Tokenizer(num_words=max_feature)
tokenize.fit_on_texts(list(X_train) + list(X_test))
X_train = tokenize.texts_to_sequences(X_train)
X_test = tokenize.texts_to_sequences(X_test)
X_train = sequence.pad_sequences(X_train, maxlen=maxlen, padding='post')
X_test = sequence.pad_sequences(X_test, maxlen=maxlen, padding='post')

In [10]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('glove.6B.50d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [12]:
word_index = tokenize.word_index
nb_words = min(max_feature, len(word_index))
embedding_matrix = np.zeros((nb_words, embedding_size))

In [14]:
# create a weight matrix for words in training docs
for word, i in word_index.items():
    if i >= max_feature: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [96]:
#Regex to remove all Non-Alpha Numeric and space
special_character_removal=re.compile(r'[^a-z\d ]',re.IGNORECASE)

#regex to replace all numerics
replace_numbers=re.compile(r'\d+',re.IGNORECASE)

In [117]:
def text_to_wordlist(text, remove_stopwords=False, stem_words=False):
    # Clean the text, with the option to remove stopwords and to stem words.
    
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
    
    text = " ".join(text)
    
    #Remove Special Characters
    text=special_character_removal.sub('',text)
    
    #Replace Numbers
    text=replace_numbers.sub('n',text)

    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)

In [94]:
from sklearn.metrics import f1_score

class f1Evaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=1, batch_size = 512)
            y_pred[y_pred>=0.1]=0
            y_pred[y_pred<=0.1]=1
            score = f1_score(self.y_val, y_pred)
            print("\n F1 - epoch: %d - score: %.6f \n" % (epoch+1, score))

In [68]:
def get_model():
    inp = Input((maxlen,))
    embed = Embedding(input_dim=max_feature, output_dim=embedding_size, weights = [embedding_matrix], trainable = True)(inp)
    x = SpatialDropout1D(0.2)(embed)
    x = Bidirectional(GRU(12, return_sequences = True))(x)
    avg = GlobalAveragePooling1D()(x)
    maxpool = GlobalMaxPooling1D()(x)
    con = concatenate([avg, maxpool])
    x = Dense(8, activation='relu')(con)
    out = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=inp, outputs=out)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [69]:
model = get_model()
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 150)           0                                            
____________________________________________________________________________________________________
embedding_2 (Embedding)          (None, 150, 50)       500000      input_2[0][0]                    
____________________________________________________________________________________________________
spatial_dropout1d_2 (SpatialDrop (None, 150, 50)       0           embedding_2[0][0]                
____________________________________________________________________________________________________
bidirectional_2 (Bidirectional)  (None, 150, 24)       4536        spatial_dropout1d_2[0][0]        
___________________________________________________________________________________________

In [70]:
X_tra, X_val, y_tra, y_val = train_test_split(X_train, y_train, train_size=0.80, random_state=233)

In [95]:
earlyStp = EarlyStopping(patience=3)
modelChePnt = ModelCheckpoint('bi_Gru.h5')
f1 = f1Evaluation(validation_data=(X_val, y_val), interval=1)

hist = model.fit(x = X_tra, y = y_tra, epochs = 5, validation_data=[X_val, y_val], batch_size=1024, callbacks=[earlyStp, modelChePnt, f1])

Train on 25569 samples, validate on 6393 samples
Epoch 1/5
6393/6393 [==============================] - 2s     1s - lo

 F1 - epoch: 1 - score: 0.126044 

25569/25569 [==============================] - 30s - loss: 0.2505 - acc: 0.9291 - val_loss: 0.2395 - val_acc: 0.9327
Epoch 2/5
6393/6393 [==============================] - 2s     1s - lo

 F1 - epoch: 2 - score: 0.126044 

25569/25569 [==============================] - 31s - loss: 0.2474 - acc: 0.9291 - val_loss: 0.2360 - val_acc: 0.9327
Epoch 3/5
6393/6393 [==============================] - 2s     1s - lo

 F1 - epoch: 3 - score: 0.126044 

25569/25569 [==============================] - 31s - loss: 0.2438 - acc: 0.9291 - val_loss: 0.2318 - val_acc: 0.9327
Epoch 4/5
6393/6393 [==============================] - 2s     1s - lo

 F1 - epoch: 4 - score: 0.126044 

25569/25569 [==============================] - 31s - loss: 0.2380 - acc: 0.9291 - val_loss: 0.2233 - val_acc: 0.9327
Epoch 5/5
6393/6393 [==============================] - 2s  

In [33]:
f1 = f1Evaluation(validation_data=(X_val, y_val), interval=1)
f1

In [23]:
y_pred = model.predict(X_test, batch_size=512, verbose=1)

17197/17197 [==============================] - 20s    


TypeError: unhashable type: 'numpy.ndarray'

In [50]:
submission = pd.read_csv('sample_submission_3Mm4cJo.csv')
submission['id'] = test.id
submission['label'] = y_pred

In [53]:
submission['label'].loc[submission['label']>=0.1]=1
submission['label'].loc[submission['label']<0.1]=0

C:\Users\dhaval_m\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [58]:
submission['label'] = submission.label.astype('int')
submission.head()

,id,label
0,31963,0
1,31964,1
2,31965,0
3,31966,0
4,31967,0


In [59]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17197 entries, 0 to 17196
Data columns (total 2 columns):
id       17197 non-null int64
label    17197 non-null int32
dtypes: int32(1), int64(1)
memory usage: 201.6 KB


In [61]:
submission.to_csv('sub.csv', index=False)

In [93]:
y_pred[y_pred>=0.1]=0
y_pred[y_pred<=0.1]=1
Counter(y_pred.astype('int')[:, 0])

Counter({1: 17197})

In [115]:
gen = 'you might be a libtard if... #libtard  #sjw #liberal #politics '

special_character_removal.sub('', replace_numbers.sub('', gen))

'you might be a libtard if libtard  sjw liberal politics '

In [105]:
blob = TextBlob('awsm')
blob.correct()

TextBlob("was")

In [106]:
import itertools

In [112]:
''.join(''.join(s)[:2] for _,s in itertools.groupby('sooo'))

'soo'

ImportError: No module named 'preprocessor'